<a href="https://colab.research.google.com/github/M-110/automate-the-boring-stuff/blob/main/17_Keeping_Time_Scheduling_Tasks_Launching_Programs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project: Super Stopwatch

In [ ]:
%%writefile stopwatch.py
#!/usr/bin/env python
import time


def main():
  last_time = time.time()
  laps = []
  print('Hit enter to start a new lap')
  try:
    while True:
      input('')
      laps.append(time.time() - last_time)
      last_time = time.time()
      print(f'Lap #{len(laps)}: {laps[-1]}')
  except KeyboardInterrupt:
    print('\nLap results:\n================')
    for i, lap in enumerate(laps, start=1):
      print(f'Lap #{i}: {lap:.2f}')


if __name__ == '__main__':
  main()


Writing stopwatch.py


In [ ]:
!chmod +x stopwatch.py

In [ ]:
!./stopwatch.py

Hit enter to start a new lap

Lap #1: 2.252493381500244

Lap #2: 1.1044199466705322

Lap #3: 1.1058635711669922

Lap #4: 0.8061566352844238

Lap #5: 0.9045369625091553

Lap #6: 0.9042949676513672

Lap #7: 0.7047913074493408

Lap #8: 0.8037741184234619

Lap #9: 0.7032372951507568

Lap #10: 0.803471565246582

Lap #11: 0.8034982681274414

Lap results:
Lap #1: 2.25
Lap #2: 1.10
Lap #3: 1.11
Lap #4: 0.81
Lap #5: 0.90
Lap #6: 0.90
Lap #7: 0.70
Lap #8: 0.80
Lap #9: 0.70
Lap #10: 0.80
Lap #11: 0.80
^C


# Threading

In [ ]:
import threading
import time

In [ ]:
def take_a_nap():
  print('Taking a nap')
  time.sleep(1)
  print('Wake up!')

thread = threading.Thread(target=take_a_nap)
thread.start()

print('End of script')

Taking a nap
End of script


# Project threaded downloading from XKCD

In [ ]:
from pathlib import Path
import time
import threading

import requests
from bs4 import BeautifulSoup


def download_all_xkcd_comics(home_page, dest='.', n=10):
  """Download all comics xkcd and save them to the destination."""
  dest = Path(dest)
  if not dest.is_dir():
    dest.mkdir()
  home = BeautifulSoup(requests.get(home_page).text)
  count = home.find('a', rel='prev')['href'].strip('/')
  download_threads = []
  for i in range(1, 200, n):
    download_thread = threading.Thread(target=download_n_xcd_comics,
                                       args=(home_page, dest, i, n))
    download_threads.append(download_thread)
  for thread in download_threads:
    thread.start()
  for thread in download_threads:
    thread.join()
  print(f'Saved 200 files to {str(dest)}')


def download_n_xcd_comics(home_page, dest, i, n):
  for j in range(n):
    html = requests.get(f'{home_page}/{i + j}').text
    page = BeautifulSoup(html)
    download_comic_from_page(page, dest, i + j)



def download_comic_from_page(page, dest, i, retry=True):
  """Download the comic image from the page to target directory."""
  try:
    img = page.find('div', id='comic').img['src']
    img_content = requests.get('http:' + img).content
    with open(dest / f"{i:04}_{img.split('/')[-1]}", 'wb') as img_file:
      img_file.write(img_content)
  except:
    if retry:
      time.sleep(1)
      print(f'Retrying downloading comic #{i}')
      download_comic_from_page(page, dest, i, retry=False)
    else:
      print(f'Failed to download comic #{i}')


In [ ]:
download_all_xkcd_comics('https://xkcd.com', 'my_comics', n=10)

Saved 200 files to my_comics


In [ ]:
!ls my_comics | wc -l

200


# Launching Other Programs from Python

In [ ]:
import subprocess

In [ ]:
!echo "Hello, World!" > hello.txt

cat: 'Hello, World!': No such file or directory


In [ ]:
subprocess.Popen(['start', 'hello.txt'], shell=True)

# Project: Simple Countdown Program

In [ ]:
%%writefile countdown.py 
#!/usr/bin/env python
"""A countdown timer that plays a sound at the end."""
import argparse
import time
import subprocess


def main():
  args = get_args()
  for i in range(args.length, 0, -1):
    print(i)
    time.sleep(1)
  print('Beep beep beep')
  subprocess.Popen(['start', 'alarm.wav'], shell=True)


def get_args():
  """Get arguments from command line"""
  parser = argparse.ArgumentParser(
    description="Create a countdown with a sound effect"
  )
  parser.add_argument('length',
                      type=int,
                      help='Countdown length in seconds.')
  return parser.parse_args()


if __name__ == '__main__':
  main()


Writing countdown.py


In [ ]:
!python countdown.py 5

5
4
3
2
1
Beep beep beep
alarm.wav: 1: alarm.wav: start: not found


# Practice Projects

## Prettified Stopwatch

In [ ]:
%%writefile pretty_stopwatch.py
#!/usr/bin/env python
"""Run a stop watch that tracks laps."""
import time


def main():
  input('Press enter to begin a new lap')
  laps = [time.time()]
  try:
    while True:
      input('')
      laps.append(time.time())
      print(f'Lap #{len(laps)-1}: {laps[-1] - laps[-2]:.2f} '
            f'({laps[-1] - laps[0]:.2f})', end='')
  except KeyboardInterrupt:
    pass


if __name__ == '__main__':
  main()
    

Overwriting pretty_stopwatch.py


In [ ]:
!python pretty_stopwatch.py

Press enter to begin a new lap

Lap #1: 1.31 (1.31)
Lap #2: 1.51 (2.81)
Lap #3: 2.11 (4.92)
Lap #4: 1.91 (6.84)
Lap #5: 1.51 (8.34)
Lap #6: 1.51 (9.85)
Lap #7: 1.01 (10.86)
Lap #8: 1.01 (11.86)
Lap #9: 0.90 (12.77)
Lap #10: 0.80 (13.57)^C


# Scheduled Comic Downloader

In [ ]:
%%writefile daily_comic_download.py
#!/usr/bin/env python
"""Downloads any new comics from websites."""
from pathlib import Path

from bs4 import BeautifulSoup
import requests


def main():
  save_dir = Path('lefthandtoons')
  save_dir.mkdir(exist_ok=True)
  html = requests.get('http://www.lefthandedtoons.com/').text
  page = BeautifulSoup(html, 'html.parser')
  img = page.find('img', class_='comicimage')['src']
  
  img_name = img.split('/')[-1]
  if img_name not in [file.name for file in save_dir.iterdir()]:
    with open(save_dir / img_name, 'wb') as img_file:
      img_content = requests.get(img).content
      img_file.write(img_content)
    print(f'Saved new comic {img_name!r} to {save_dir.name} directory')
  else:
    print('No new comics found')

if __name__ == '__main__':
  main()


Overwriting daily_comic_download.py


In [ ]:
!python daily_comic_download.py

Saved new comic 'drew_ariotheory.gif' to lefthandtoons directory


In [ ]:
!python daily_comic_download.py

No new comics found
